In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import datasets, models, transforms

import numpy as np
import matplotlib.pyplot as plt

import time
import os

import pandas as pd
from utils import * 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object

#clear cuda memory
torch.cuda.empty_cache()

### Inspect the inference time of the model

In [2]:
transforms_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(), # data augmentation
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # normalization
])

transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_dir = './CelebA_HQ_facial_identity_dataset/'
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transforms_train)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transforms_test)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=12, pin_memory=True if torch.cuda.is_available() else False )
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=True if torch.cuda.is_available() else False )

print('Train dataset size:', len(train_dataset))
print('Test dataset size:', len(test_dataset))

class_names = train_dataset.classes
len(class_names)

Train dataset size: 4359
Test dataset size: 1248


310

In [3]:
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 310) # multi-class classification (num_of_class == 310)
model.load_state_dict(torch.load('models/model_310.pth',map_location=device))
model.to(device)

/opt/anaconda3/envs/CSI4133/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/CSI4133/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
model.eval()

criterion = nn.CrossEntropyLoss()

with torch.no_grad():
    running_loss = 0.
    running_corrects = 0


    for i, (inputs, labels) in enumerate(test_dataloader):
        start_time = time.time()
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        print('Batch {} / {} - Time: {:.2f} sec'.format(i+1, len(test_dataloader), time.time() - start_time))
        loss = criterion(outputs, labels)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

        # if i == 0:
        #     print('[Prediction Result Examples]')
        #     images = torchvision.utils.make_grid(inputs[:4])
        #     imshow(images.cpu(), title=[class_names[x] for x in labels[:4]])
        #     images = torchvision.utils.make_grid(inputs[4:8])
        #     imshow(images.cpu(), title=[class_names[x] for x in labels[4:8]])
        #     images = torchvision.utils.make_grid(inputs[8:12])
        #     imshow(images.cpu(), title=[class_names[x] for x in labels[8:12]])
        #     images = torchvision.utils.make_grid(inputs[12:16])
        #     imshow(images.cpu(), title=[class_names[x] for x in labels[12:16]])

    epoch_loss = running_loss / len(test_dataset)
    epoch_acc = running_corrects / len(test_dataset) * 100


Batch 1 / 10 - Time: 2.79 sec
Batch 2 / 10 - Time: 2.75 sec
Batch 3 / 10 - Time: 2.80 sec
Batch 4 / 10 - Time: 2.69 sec
Batch 5 / 10 - Time: 2.58 sec
Batch 6 / 10 - Time: 2.64 sec
Batch 7 / 10 - Time: 2.28 sec
Batch 8 / 10 - Time: 2.22 sec
Batch 9 / 10 - Time: 2.29 sec
Batch 10 / 10 - Time: 1.73 sec


In [23]:
epoch_acc

tensor(81.3301)

In [4]:
import json
f = open('models/class_names_310.json' , 'r')
classNames = json.load(f)
f.close()


In [6]:
def pgd(model, X, y, epsilon, alpha, num_iter):

    delta = torch.rand_like(X, requires_grad=True)
    #set delta to be in the range of perturbation
    delta.data = delta.data * 2 * epsilon - epsilon

    for t in range(num_iter):
        
        yd = model(X + delta)
        loss = nn.CrossEntropyLoss()(yd, y)
        loss.backward()
        delta.data = (delta + alpha*delta.grad.detach().sign()).clamp(-epsilon,epsilon)
        delta.grad.zero_()

        
    return delta.detach()

In [14]:
from PIL import Image
import cv2
# test the model with a new image
img = cv2.imread('CelebA_HQ_facial_identity_dataset/test/Xiang/0000.jpg')
# convert to RGB
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = Image.fromarray(img)
img = transforms_test(img)
img = img.unsqueeze(0)
img = img.to(device)

model.eval()

start = time.time()
outputs = model(img)
_, preds = torch.max(outputs, 1)

# pgd attack

delta = pgd(model, img, preds, 0.1, 0.01, 1)
img_pgd = img + delta
outputs = model(img_pgd)
_, preds = torch.max(outputs, 1)
print('Time: {:.2f} sec'.format(time.time() - start))


print('Predicted class: {}'.format(classNames[preds.item()]))


Time: 0.19 sec
Predicted class: Bowen


In [25]:
#Evaluate all Aaron_Taylor-Johnson images
import os
import cv2
import time
import json
import torch
import torchvision
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

f = open('models/class_names_310.json' , 'r')
classNames = json.load(f)
f.close()

model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 310) # multi-class classification (num_of_class == 310)
model.load_state_dict(torch.load('models/model_310.pth',map_location=device))
model.to(device)

model.eval()

transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def predict(img):
    img = transforms_test(img)
    img = img.unsqueeze(0)
    img = img.to(device)
    outputs = model(img)
    _, preds = torch.max(outputs, 1)
    return classNames[preds.item()]


def evaluate(path):
    correct = 0
    total = 0
    for filename in os.listdir(path):
        img = cv2.imread(os.path.join(path,filename))
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        #visualize the image
        #plt.imshow(img)

        label = predict(img)
        if label == 'Jiaxun':
            correct += 1
        total += 1
    print(total, correct)
    return correct/total

evaluate('CelebA_HQ_facial_identity_dataset/test/Jiaxun')

/opt/anaconda3/envs/CSI4133/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/CSI4133/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


11 11


1.0

In [15]:
class_names[64]

'Bowen'

In [19]:
torch.Tensor([64]).type(torch.LongTensor)

tensor([64])